## Converts .eeg file to pandas dataframe

In [8]:
import os
import numpy as np
import struct

def avg_to_ascii(file_name, chanlist='all', triallist='all', typerange='all', accepttype='all', rtrange='all', responsetype='all', data_format='auto'):
    """This function reads the data from a binary EEG file, extracts and scales the data, and returns it in ASCII format.

    Parameters:

    - **file_name (str)**: The name of the binary EEG file you want to read.
    - **chanlist (str or list)**: List of channels to read from the file. Default is 'all', which reads all channels in the file.
    - **triallist (str or list)**: List of trials to read from the file. Default is 'all', which reads all trials in the file.
    - **typerange (str, list or tuple)**: Range of type codes to read from the file. Default is 'all', which reads all type codes in the file.
    - **accepttype (str, list or tuple)**: Range of accept codes to read from the file. Default is 'all', which reads all accept codes in the file.
    - **rtrange (str, list or tuple)**: Range of reaction times to read from the file. Default is 'all', which reads all reaction times in the file.
    - **responsetype (str, list, or tuple)**: Range of response codes to read from the file. Default is 'all', which reads all response codes in the file.
    - **data_format (str)**: The format in which to return the data. Default is 'auto', which returns the data in its native format.

    Returns:

    - **data (numpy.ndarray)**: A 3D array containing the EEG data (in volts) for each trial, channel, and time point.
    - **chan_names (list of str)**: A list of channel names.
    - **rate (int)**: The sample rate (in Hz) of the EEG data.
    - **xmin (float)**: The minimum time (in seconds) of the data."""

    #Check if file ends with .eeg
    if not file_name.endswith('.avg'):
        raise ValueError("File must be a binary avg file (.avg).")

    if not os.path.isfile(file_name):
        raise ValueError(f"File {file_name} not found.")
    
    with open(file_name, 'rb') as f:
        try:
            # Read general part of the ERP header and set variables
            f.read(20) # skip revision number
            f.read(342) # skip the first 362 bytes

            nsweeps = struct.unpack('<H', f.read(2))[0]  # number of sweeps
            f.read(4)  # skip 4 bytes
            pnts = struct.unpack('<H', f.read(2))[0]  # number of points per waveform
            chan = struct.unpack('<H', f.read(2))[0]  # number of channels
            f.read(4)  # skip 4 bytes
            rate = struct.unpack('<H', f.read(2))[0]  # sample rate (Hz)
            f.read(127)  # skip 127 bytes
            xmin = struct.unpack('<f', f.read(4))[0]  # in s
            xmax = struct.unpack('<f', f.read(4))[0]  # in s
            f.read(387)  # skip 387 bytes

            # Read electrode configuration
            chan_names = []
            baselines = []
            sensitivities = []
            calibs = []
            factors = []
            for elec in range(chan):
                chan_name = f.read(10).decode('ascii').strip('\x00')
                chan_names.append(chan_name)
                f.read(37)  # skip 37 bytes
                baseline = struct.unpack('<H', f.read(2))[0]
                baselines.append(baseline)
                f.read(10)  # skip 10 bytes
                sensitivity = struct.unpack('<f', f.read(4))[0]
                sensitivities.append(sensitivity)
                f.read(8)  # skip 8 bytes
                calib = struct.unpack('<f', f.read(4))[0]
                calibs.append(calib)
                factor = calib * sensitivity / 204.8
                factors.append(factor)

        except struct.error:
            raise ValueError("Error reading binary file. File may be corrupted or not in the expected format.")
        except Exception as e:
            raise ValueError(f"Error reading file: {e}")

    # Read and process epoch datapoints data
    data = np.empty((nsweeps, len(chan_names), pnts), dtype=float)
    sweep_headers = []

    # Constants for the sweep header size in bytes and data point size in bytes
    SWEEP_HEAD_SIZE = 13
    DATA_POINT_SIZE = 4

    with open(file_name, 'rb') as f:
        # Ensure the file pointer is at the beginning of the EEG data
        f.seek((900 + chan * 75))

        for point in range(pnts):
            for channel in range(chan):
                try:
                    # Read the data point as a 4-byte integer
                    value = struct.unpack('<l', f.read(DATA_POINT_SIZE))[0]

                    # Scale the data point to microvolts and store it in the data array
                    data[channel, point] = value * factors[channel] / nsweeps
                except struct.error:
                    raise ValueError("Error reading data points. File may be corrupted or not in the expected format.")
                except Exception as e:
                    raise ValueError(f"Error reading data points: {e}")

    #Convert data from microvolts to volts
    data = data * 1e-6
    # Return relevant data in ASCII format
    return data, chan_names, rate, xmin

## Example Usage

In [10]:
file_name = "/home/woess/current assignment/read_neuro_scratch/16017_rest_ec.eeg"


eeg_data, channels, sample_rate, xmix = eeg_to_ascii(file_name)
print("Channels:", channels)
print("EEG data:", eeg_data)
print("Sample rate:", sample_rate)
print("Xmin:", xmix)

Channels: ['O2', 'O1', 'Pz', 'P4', 'P8', 'C4', 'T8', 'P7', 'P3', 'Cz', 'C3', 'Fz', 'F4', 'F8', 'T7', 'F3', 'Fp2', 'F7', 'Fp1', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'Oz', 'M2', 'EKG', 'HEOG', 'VEOG', 'Startle']
EEG data: [[[-1.03507472e-06 -1.25362509e-06 -1.58189545e-06 ...  1.01407965e-05
    8.57847270e-06  7.81176716e-06]
  [ 2.87407081e-06  3.38130338e-06  4.04318185e-06 ...  1.28498425e-05
    1.03706154e-05  8.09051526e-06]
  [ 6.74066277e-06  5.77097524e-06  4.74909658e-06 ... -1.01304176e-05
   -1.03525264e-05 -9.99222967e-06]
  ...
  [ 4.14800892e-06  3.89417023e-06  3.63603170e-06 ...  4.58451658e-06
    4.54774556e-06  4.55174885e-06]
  [ 1.13392650e-05  1.13626917e-05  1.13747016e-05 ... -3.82765619e-05
   -3.81490495e-05 -3.79926243e-05]
  [-3.25453231e-07 -2.26556964e-07 -6.07908085e-08 ... -1.73476210e-07
   -3.86392310e-07 -5.17611494e-07]]

 [[ 7.89465024e-06  8.18941153e-06  8.21135552e-06 ... -1.90112133e-06
   -3.50169884e-06 -6.0713

In [11]:
#Place eeg_data into a pandas dataframe
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(eeg_data[0].transpose(), columns=channels)
#Display decimals (change '10' to the number of decimals you want to display)
pd.set_option('display.float_format', lambda x: '%.50f' % x)
df

# #plot the data
# plt.plot(df.index.to_list(),df["O2"].to_list())
#plt.show()

,O2,O1,Pz,P4,P8,C4,T8,P7,P3,Cz,...,CP3,CPz,CP4,TP8,Oz,M2,EKG,HEOG,VEOG,Startle
0,-0.00000103507471729244567052407016011228435559...,0.000002874070809339173197366175752098804707657...,0.000006740662770025665044288775556990955806213...,0.000000154794156260322777909319317754532896458...,-0.00000246217601423268199749717216606725855854...,0.000003829969205326051526888913451029949897019...,0.000025461562921118456657813833299286443434539...,0.000001892966468367376230055431490795214699573...,0.000010256743825157173353058298603812659166578...,0.000010105952966041513245952498945001707397750...,...,0.000015309201362251768224753689651151944417506...,0.000008894288363590021918034592141033556345064...,0.000002736179463245207453591170687623446156067...,0.000002539128281568991799178490118005235842701...,0.000001379210001468891211637421076452447721294...,-0.00000044911063152540007352905566777312440507...,-0.00000191550354858918582065942064207586525981...,0.000004148008922929875202728015187725318924094...,0.000011339265027127111995779336350853583326170...,-0.00000032545323083468247552302505566645951518...
1,-0.00000125362508733815037759433842073919507242...,0.000003381303384594502908284301481711686676590...,0.000005770975239429389759223711087177122180946...,-0.00000019052729003306010868477946961635494105...,-0.00000268472968142305006358522972975677589602...,0.000004264401080654351470328958934041807538051...,0.000016572167476260801961557983519313097531266...,0.000002724614382605068249127095239559004369311...,0.000010140648207961930435828251662044863223854...,0.000009972657998150680103035918555587358014236...,...,0.000015456137194487381224510674782024466367147...,0.000008399362220297916905090152006607695511775...,0.000002388633962982566736509481772343654881751...,0.000001399523027721443051163158904914851632383...,0.000001457348430922138438208825704511806264918...,-0.00000069479445999604649560570492419775412429...,-0.00000193685446669405777461508630366004979350...,0.000003894170229905284487457832465207019367881...,0.000011362691728936624602111331305831498639236...,-0.00000022655696433503179568276129050852230051...
2,-0.00000158189545320055915782198052710327829117...,0.000004043181845845537716074673262323102562731...,0.000004749096575687872014159569139524208480906...,-0.00000079976980734500097508185047223694397189...,-0.00000294479572556156188895145307815592872202...,0.000004701057009951910011880202994172250896554...,0.000006485786185148754214588665001350875627394...,0.000003863923095923382799738582393800356840074...,0.000009928918270088615233661712144996869255919...,0.000009783316870234557392771446160395498736761...,...,0.000015763649723303388774930944293828360969200...,0.000007939131318926229082021464589224990504590...,0.000002000017599420971044270551111865508175924...,0.000000168731561134336514516212897699110051519...,0.000001604877344216220044531735766657476460750...,0.000000119950644075288436392085367893090008806...,-0.00000197466338417143573129709614721871702158...,0.000003636031699206796215766016380777081451469...,0.000011374701620370614255648225587869148967001...,-0.00000006079080849303863163352826953778063057...
3,-0.00000175329587909800466658265163250041140940...,0.000004977581053462927416456296897395716882783...,0.000003847613366815494319230633868400559549627...,-0.00000145734843092213843820882570451180626491...,-0.00000302901323586411291515873948909653989858...,0.000004907004407505155400056467973612939204031...,-0.00000156113761615415559867400298321715723659...,0.000005173297802757588335518731947892945299827...,0.000009837435516819823406444930902203793721128...,0.000009588638012792215741105991688719711874000...,...,0.000016145149114163357800246986761827372447442...,0.000007518488578207324823574324995645312696979...,0.000001651137666776776227113327118189989306529...,-0.00000085952272398571949228545090865294397985...,0.000001943230088072595983413048958965418933075...,0.000001284317032113903996694065021144659510810...,-0.000002040050570867